In [1]:
import pandas as pd

In [2]:
INDEX = "MS50"
df = pd.read_csv( f"./data/{INDEX}/news_D.csv" )

In [3]:
df["date"] = pd.to_datetime(df["date"], errors="coerce", utc=True)
df["date"] = df["date"].dt.tz_convert(None)
df["date"] = df["date"].dt.floor("D")
df = df.dropna(subset=["date", "ticker"])

In [4]:
#weekly = ( df.groupby( ["ticker", pd.Grouper(key="date", freq="W-SUN")] ).size().rename("news_count").reset_index() )
daily = (
    df.groupby(["ticker", "date"])
        .size()
        .reset_index(name="news_count")
        .sort_values(["ticker","date"])
        .reset_index(drop=True)
)

In [5]:
wide = daily.pivot(index="date", columns="ticker", values="news_count")
full_days = pd.date_range(start=pd.to_datetime("2020-01-01"), end=df["date"].max().normalize(), freq="D")
wide = wide.reindex(full_days)
wide.index.name = "date"
wide = wide.fillna(0).astype("int64")
daily_full = (
    wide.stack(dropna=False)
        .rename("news_count")
        .reset_index()
        .rename(columns={"level_1":"ticker"})
        .sort_values(["ticker","date"])
        .reset_index(drop=True)
)

C:\Users\Lika44\AppData\Local\Temp\ipykernel_24572\3960079796.py:7: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  wide.stack(dropna=False)


In [6]:
daily_full.to_csv( f"./data/{INDEX}/news_volume_D.csv", index=False )